In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
from transformers import (AutoTokenizer, AutoConfig,
                              AutoModelForSequenceClassification, TrainingArguments, Trainer)

In [5]:
model_checkpoint="bert-base-uncased"

In [ ]:
#model_checkpoint="biobert-base-uncased"

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [8]:
from transformers import DataCollatorForTokenClassification


data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [9]:
df0=pd.DataFrame(columns=["name","tokens"])

In [10]:
df_tr0=pd.DataFrame(columns=["name","tokens"])

In [20]:
import spacy 
nlp = spacy.load("en_core_web_lg")

In [21]:
for file in os.listdir("/kaggle/input/annotated/copious_published/dev"):
    if(file.endswith('.txt') or file.endswith('..txt')):
        
        texts = open("/kaggle/input/annotated/copious_published/dev/"+file,"r")

        sentence_tokens=[]

        for doc in nlp.pipe(texts):
            sentence_tokens.append([[token.text for token in sent] for sent in doc.sents])
        dic={"name":[file],"tokens":[sentence_tokens]}
        

        df0=pd.concat([df0,pd.DataFrame(dic)],ignore_index=True)
        
        


In [22]:
df0

,name,tokens
0,104421_English_33451118_1922-26..txt,"[[[44, ENUMERATION, OF, PHILIPPINE, PLANTS, \n..."
1,28549848.txt,"[[[—, 324, —, Fruit, one, -, seeded, by, abort..."
2,34179045.txt,"[[[Anisoptera, ], XVII, ., DIPTEEOCARPACEAE, 6..."
3,11364519.txt,"[[[ , 22, J., Perkins, :, Dipterocarpaceae, by..."
4,109841_English_35421956_1906.txt,"[[[AUTOPSIES, AT, THE, PHILIPPINE, MEDICAL, SC..."
...,...,...
62,109841_English_35421830_1906.txt,"[[[ILLUSTRATIONS, ., \n]], [[Plates, I, and, I..."
63,104481_English_33474555_1822.txt,"[[[146, GRAKLE, ., \n]], [[Mina, ,, or, Hill, ..."
64,111121_English_36025302_1908.txt,"[[[GEOLOGIC, RECONNAISSANCE, OF, MINDANAO, AND..."
65,109841_English_35421952_1906.txt,"[[[ANIMAL, PARASITES, OF, MAN, .], [209, \n]],..."


In [12]:
pip install mendelai-brat-parser

  Preparing metadata (setup.py) ... done
  Created wheel for mendelai-brat-parser: filename=mendelai_brat_parser-0.0.11-py3-none-any.whl size=4920 sha256=1250e04894a104f67130003feeeb67bf0547ed15f223ac306064a8456f2e9c42
  Stored in directory: /root/.cache/pip/wheels/66/6d/54/75653ad4624c60f22e21173848881fdf2af79baafab35e13a9
Successfully built mendelai-brat-parser
Note: you may need to restart the kernel to use updated packages.


In [15]:
from brat_parser import get_entities_relations_attributes_groups

#entities, relations, attributes, groups = get_entities_relations_attributes_groups("/kaggle/input/annotated/copious_published/train/100785_English_120409_32367238_1954.ann")

In [16]:
df1=pd.DataFrame(columns=["name","ann_text","ann_tag"])

In [17]:
df2=pd.DataFrame(columns=["name","ann_text","ann_tag"])

In [18]:
dic={}
for file in os.listdir("/kaggle/input/annotated/copious_published/dev"):
    if(file.endswith('..ann') or file.endswith('.ann')):
        

        entities, relations, attributes, groups = get_entities_relations_attributes_groups("/kaggle/input/annotated/copious_published/dev/"+file)
        l=[]
        l1=[]
        for i in entities.keys():
            a1=entities[i]
            b1=a1.text
            c1=a1.type
            l.append(b1)
            l1.append(c1)
        dic={"name":[file],"ann_text":[l],"ann_tag":[l1]}
        df_temp=pd.DataFrame(dic)
        
        df2=pd.concat([df2,df_temp],ignore_index=True)
        

In [26]:
df2

,name,ann_text,ann_tag
0,34179045.ann,"[DIPTEEOCARPACEAE, A. glabra, A. glabra, Kurz,...","[Taxon, Taxon, Taxon, Taxon, GeographicalLocat..."
1,37190474.ann,"[Malayan Peninsula, D. laevis, D. tuberculatus...","[GeographicalLocation, Taxon, Taxon, Taxon, Ta..."
2,36442110.ann,"[Dipterocarpaceae, Shorea, tropical forests, M...","[Taxon, Taxon, Habitat, GeographicalLocation, ..."
3,28831271.ann,"[Malesia, Philippines, Zamboanga, Mindanao, Va...","[GeographicalLocation, GeographicalLocation, G..."
4,109841_English_35421888_1906.ann,"[Japan, Oshima, Riukiu Islands, Tokyo, Japan, ...","[GeographicalLocation, GeographicalLocation, G..."
...,...,...,...
62,107004_English_78294_34354191_1912.ann,"[PHILIPPINE ISLANDS, Sus inconstans, Sus incon...","[GeographicalLocation, Taxon, Taxon, Taxon, Ge..."
63,104421_English_33451123_1922-26..ann,"[Luzon, GRAMINEAE, Abra, Ilocos Norte, Pangasi...","[GeographicalLocation, Taxon, GeographicalLoca..."
64,107004_English_78294_34354195_1912.ann,"[PHILIPPINE ISLANDS, Mariquina, USSA GUEVARANU...","[GeographicalLocation, GeographicalLocation, T..."
65,111121_English_36025302_1908.ann,"[G. F. Becker, Abella, Mindanao, Mindanao, Min...","[Person, Person, GeographicalLocation, Geograp..."


In [23]:
df_dev=pd.DataFrame(columns=["tokens","tag"])

In [ ]:
df2

In [24]:
import string
y=list(string.punctuation)
y.append('\n')
for i in range(len(df0)):
    flattened_list=[]
    n=(df0["tokens"].iloc[i,])
    flattened_list = [word for sublist in n for sentence in sublist for word in sentence]
    flattened_list= [x for x in flattened_list if x not in y]
    df0["tokens"].iloc[i,]=flattened_list
    

In [67]:
df0

,name,tokens
0,104421_English_33451118_1922-26..txt,"[44, ENUMERATION, OF, PHILIPPINE, PLANTS, 1922..."
1,28549848.txt,"[—, 324, —, Fruit, one, seeded, by, abortion, ..."
2,34179045.txt,"[Anisoptera, XVII, DIPTEEOCARPACEAE, 67, 1, A...."
3,11364519.txt,"[ , 22, J., Perkins, Dipterocarpaceae, by, J.,..."
4,109841_English_35421956_1906.txt,"[AUTOPSIES, AT, THE, PHILIPPINE, MEDICAL, SCHO..."
...,...,...
62,109841_English_35421830_1906.txt,"[ILLUSTRATIONS, Plates, I, and, II, Houses, in..."
63,104481_English_33474555_1822.txt,"[146, GRAKLE, Mina, or, Hill, Moina, It, proba..."
64,111121_English_36025302_1908.txt,"[GEOLOGIC, RECONNAISSANCE, OF, MINDANAO, AND, ..."
65,109841_English_35421952_1906.txt,"[ANIMAL, PARASITES, OF, MAN, 209, intestinal, ..."


In [68]:
df_dev

,tokens,tag


In [ ]:
#df_dev["tokens"].iloc[0,]

In [ ]:

list(string.punctuation)

In [25]:
for i in range(len(df0)):
    for j in range(len(df2)):
        if df0.iloc[i]["name"].split('.txt') == df2.iloc[j]["name"].split('.ann'):
            p = 0
            a = []
            l = []

            for k in df0.iloc[i]["tokens"]:
                matched = False

                # Check if the token exists in df2
                if k in df2.iloc[j]["ann_text"]:
                    b = k
                    a.append(b)
                    l.append(df2.iloc[j]["ann_tag"][df2.iloc[j]["ann_text"].index(b)])
                    matched = True

                # Check for longer sequences
                for n in range(1, 8):
                    if p <= len(df0.iloc[i]["tokens"]) - n:
                        seq = " ".join(df0.iloc[i]["tokens"][p:p + n])
                        if seq in df2.iloc[j]["ann_text"]:
                            a.append(seq)
                            
                            l.append(df2.iloc[j]["ann_tag"][df2.iloc[j]["ann_text"].index(seq)])
                            matched = True
                        

                # If no match was found, add 'O'
                if not matched:
                    a.append(k)
                    l.append('O')

                p += 1

            dic = {"tokens": [a], "tag": [l]}
            df_dev = pd.concat([df_dev,pd.DataFrame(dic)],ignore_index=True)


In [27]:
df_dev

,tokens,tag
0,"[44, ENUMERATION, OF, PHILIPPINE, PLANTS, 1922...","[O, O, O, O, O, O, O, O, Taxon, Taxon, O, Taxo..."
1,"[—, 324, —, Fruit, one, seeded, by, abortion, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
2,"[Anisoptera, XVII, DIPTEEOCARPACEAE, DIPTEEOCA...","[O, O, Taxon, Taxon, O, O, Taxon, O, O, O, O, ..."
3,"[ , 22, J., Perkins, Perkins, Dipterocarpaceae...","[O, O, O, Person, Person, Taxon, Taxon, O, O, ..."
4,"[AUTOPSIES, AT, THE, PHILIPPINE, MEDICAL, SCHO...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
...,...,...
62,"[ILLUSTRATIONS, Plates, I, and, II, Houses, in...","[O, O, O, O, O, O, O, O, GeographicalLocation,..."
63,"[146, GRAKLE, Mina, Mina, or, Hill Moina, Moin...","[O, O, Taxon, Taxon, O, Taxon, O, O, O, O, O, ..."
64,"[GEOLOGIC, RECONNAISSANCE, OF, MINDANAO, MINDA...","[O, O, O, GeographicalLocation, GeographicalLo..."
65,"[ANIMAL, PARASITES, OF, MAN, 209, intestinal, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


In [35]:

m=pd.DataFrame(columns=df_dev.columns)

In [36]:
for i in range(len(df_dev)):
    tokens_list = []  # List to store tokens for the current row
    tags_list = [] 
    
    for k in range(len(df_dev["tokens"].iloc[i,])):
        
        
        tokens = df_dev["tokens"].iloc[i,][k].split()  # Split the tokens into individual words
        tag = df_dev["tag"].iloc[i,][k]
        for j, token in enumerate(tokens):
                if j == 0:
                    b = 'B-' + tag
                else:
                    b = 'I-' + tag

                # Append the token and tag to the modified DataFrame
                tokens_list.append(token)
                tags_list.append(b)
                tags_list1=[x.replace('B-O','O') for x in tags_list]
                

        
        
           
    m = pd.concat([m,pd.DataFrame({'tokens': [tokens_list], 'tag': [tags_list1]})], ignore_index=True)
                
    









In [37]:
m

,tokens,tag
0,"[44, ENUMERATION, OF, PHILIPPINE, PLANTS, 1922...","[O, O, O, O, O, O, O, O, B-Taxon, B-Taxon, O, ..."
1,"[—, 324, —, Fruit, one, seeded, by, abortion, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
2,"[Anisoptera, XVII, DIPTEEOCARPACEAE, DIPTEEOCA...","[O, O, B-Taxon, B-Taxon, O, O, B-Taxon, I-Taxo..."
3,"[22, J., Perkins, Perkins, Dipterocarpaceae, D...","[O, O, B-Person, B-Person, B-Taxon, B-Taxon, O..."
4,"[AUTOPSIES, AT, THE, PHILIPPINE, MEDICAL, SCHO...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
...,...,...
62,"[ILLUSTRATIONS, Plates, I, and, II, Houses, in...","[O, O, O, O, O, O, O, O, B-GeographicalLocatio..."
63,"[146, GRAKLE, Mina, Mina, or, Hill, Moina, Moi...","[O, O, B-Taxon, B-Taxon, O, B-Taxon, I-Taxon, ..."
64,"[GEOLOGIC, RECONNAISSANCE, OF, MINDANAO, MINDA...","[O, O, O, B-GeographicalLocation, B-Geographic..."
65,"[ANIMAL, PARASITES, OF, MAN, 209, intestinal, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


In [ ]:
b

In [ ]:
df_dev

In [30]:
dic={}
for file in os.listdir("/kaggle/input/annotated/copious_published/train"):
    if(file.endswith('..ann') or file.endswith('ann')):
        

        entities, relations, attributes, groups = get_entities_relations_attributes_groups("/kaggle/input/annotated/copious_published/train/"+file)
        l=[]
        l1=[]
        for i in entities.keys():
            a=entities[i]
            b=a.text
            c=a.type
            l.append(b)
            l1.append(c)
        dic={"name":[file],"ann_text":[l],"ann_tag":[l1]}
        
        df1=pd.concat([df1,pd.DataFrame(dic)],ignore_index=True)
        
    

In [75]:
df1

,name,ann_text,ann_tag
0,61098_English_18402241_unknown.ann,"[PHILIPPINES, gardens, F. vulgar -e, C. sativu...","[GeographicalLocation, Habitat, Taxon, Taxon, ..."
1,28549694.ann,"[Hopea longifolia, Hopea longifolia, Fl., Tinn...","[Taxon, Taxon, GeographicalLocation, Geographi..."
2,11298932.ann,"[Dipterocarpaceae, Doona, Moist low country, S...","[Taxon, Taxon, Habitat, GeographicalLocation, ..."
3,42271_English_11364594_1904-1905..ann,"[Cabunganan, Philippine Islands, Pliacelophryi...","[GeographicalLocation, GeographicalLocation, T..."
4,41714363.ann,"[plantain, Tahan River, dense jungle, Tembelin...","[Taxon, GeographicalLocation, Habitat, Geograp..."
...,...,...,...
529,98578_English_31594032_1983.ann,"[Philippines, Plexaura homomalla, Estuarine, I...","[GeographicalLocation, Taxon, Habitat, Geograp..."
530,41827931.ann,"[DIPTEROCABPS, Johore, Kuala Sembrong, October...","[Taxon, GeographicalLocation, GeographicalLoca..."
531,38627196.ann,"[SHOREA ROBUSTA, Dipterocarpeae, SHOREA, SHORE...","[Taxon, Taxon, Taxon, Taxon, Taxon, Taxon, Hab..."
532,42988137.ann,"[DIPTEROCARPEAE, Ancistrocladus, A. Heyneanus,...","[Taxon, Taxon, Taxon, Taxon, Taxon, Taxon, Geo..."


In [31]:
for file in os.listdir("/kaggle/input/annotated/copious_published/train"):
    if(file.endswith('.txt') or file.endswith('..txt')):
        
        texts = open("/kaggle/input/annotated/copious_published/train/"+file,"r")

        sentence_tokens=[]

        for doc in nlp.pipe(texts):
            sentence_tokens.append([[token.text for token in sent] for sent in doc.sents])
        dic={"name":[file],"tokens":[sentence_tokens]}

        df_tr0=pd.concat([df_tr0,pd.DataFrame(dic)],ignore_index=True)
        

In [32]:
y=list(string.punctuation)
y.append('\n')
for i in range(len(df_tr0)):
    flattened_list=[]
    n=(df_tr0["tokens"].iloc[i,])
    flattened_list = [word for sublist in n for sentence in sublist for word in sentence]
   

    flattened_list= [x for x in flattened_list if x not in y]
    
    df_tr0["tokens"].iloc[i,]=flattened_list
    

In [34]:
df_tr0

,name,tokens
0,44148_English_12096466_unknown.txt,"[32, ME, p., H., CARPENTER, OX, THE, GENUS, AC..."
1,39410_English_10262308_1954.txt,"[season, it, gradually, becomes, more, dispers..."
2,99366_English_31853754_1956.txt,"[58, COMMERCIAL, FISHERIES, REVIEW, Vol, 18, N..."
3,38627554.txt,"[VATERIA, MALABARICA, Nat, order, Dipterocarpe..."
4,694855.txt,"[188, The, Philippine, Journal, of, Science, w..."
...,...,...
529,35334_English_8689460_unknown.txt,"[from, the, Philippine, Islands, 97, Tabanidae..."
530,692399.txt,"[504, The, Philippine, Journal, of, Science, 1..."
531,119097_English_38679200_2008.txt,"[Section, 1, Introduction, Table, 1, Examples,..."
532,41827930.txt,"[276, NOTES, ON, DIPTEKOCARPS, low, and, level..."


In [33]:
df_train=pd.DataFrame(columns=df_dev.columns)
n=pd.DataFrame(columns=df_train.columns)

In [39]:
for i in range(len(df_tr0)):
    for j in range(len(df1)):
        if(df_tr0.iloc[i,]["name"].split('.txt')==df1.iloc[j,]["name"].split('.ann')):
            p = 0
            a = []
            l = []

            for k in df_tr0.iloc[i]["tokens"]:
                matched = False

                # Check if the token exists in df2
                if k in df1.iloc[j]["ann_text"]:
                    b = k
                    a.append(b)
                    l.append(df1.iloc[j]["ann_tag"][df1.iloc[j]["ann_text"].index(b)])
                    matched = True

                # Check for longer sequences
                for n in range(1, 8):
                    if p <= len(df_tr0.iloc[i]["tokens"]) - n:
                        seq = " ".join(df_tr0.iloc[i]["tokens"][p:p + n])
                        if seq in df1.iloc[j]["ann_text"]:
                            a.append(seq)
                            l.append(df1.iloc[j]["ann_tag"][df1.iloc[j]["ann_text"].index(seq)])
                            matched = True
                        

                # If no match was found, add 'O'
                if not matched:
                    a.append(k)
                    l.append('O')

                p += 1

            dic = {"tokens": [a], "tag": [l]}
            df_train = pd.concat([df_train,pd.DataFrame(dic)],ignore_index=True)


In [41]:
from tqdm import tqdm

In [40]:
df_train

,tokens,tag
0,"[32, ME, p., H., CARPENTER OX, OX, THE, GENUS,...","[O, O, O, O, Taxon, O, O, O, Taxon, Taxon, O, ..."
1,"[season, it, gradually, becomes, more, dispers...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
2,"[58, COMMERCIAL, FISHERIES, REVIEW, Vol, 18, N...","[O, O, O, O, O, O, O, O, O, O, O, Geographical..."
3,"[VATERIA, VATERIA, VATERIA MALABARICA, MALABAR...","[Taxon, Taxon, Taxon, O, O, O, Taxon, Taxon, O..."
4,"[188, The, Philippine, Journal, of, Science, w...","[O, O, O, O, O, O, O, O, Taxon, O, O, O, O, O,..."
...,...,...
529,"[from, the, Philippine Islands, Islands, 97, T...","[O, O, GeographicalLocation, O, O, Taxon, Taxo..."
530,"[504, The, Philippine, Journal, of, Science, 1...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
531,"[Section, 1, Introduction, Table, 1, Examples,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
532,"[276, NOTES, ON, DIPTEKOCARPS, DIPTEKOCARPS, l...","[O, O, O, Taxon, Taxon, O, O, O, O, O, O, O, O..."


In [42]:
df_train=df_train.drop(116)

In [44]:
n=pd.DataFrame(columns=["tokens","tag"])

In [45]:
n

,tokens,tag


In [46]:
for i in range(len(df_train)):
    tokens_list = []  # List to store tokens for the current row
    tags_list = [] 
    for k in range(len(df_train["tokens"].iloc[i,])):
        
        
        tokens = df_train["tokens"].iloc[i,][k].split()  # Split the tokens into individual words
        tag = df_train["tag"].iloc[i,][k]
        for j, token in enumerate(tokens):
                if j == 0:
                    b = 'B-' + tag
                else:
                    b = 'I-' + tag

                # Append the token and tag to the modified DataFrame
                tokens_list.append(token)
                tags_list.append(b)
                tags_list1=[x.replace('B-O','O') for x in tags_list]

        
       
            
    n = pd.concat([n,pd.DataFrame({'tokens': [tokens_list], 'tag': [tags_list1]})], ignore_index=True)
                
    









In [116]:
len(n)==len(df_train)

True

In [47]:
def tokenize_and_align_labels(examples,tokenizer):
    tokenized_inputs = tokenizer(examples["tokens"].tolist(), truncation=True, max_length=512,is_split_into_words=True)
    
    all_labels = examples["tag"].tolist()
    new_labels = []
    tokenized_data = []
    
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))
        #tokenized_data.append({"input_ids": tokenized_inputs['input_ids'][i], "attention_mask": tokenized_inputs['attention_mask'][i], "labels": labels})

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [48]:
columns_to_remove = ['name']
df1= df1.drop(columns_to_remove, axis=1)
df2= df2.drop(columns_to_remove, axis=1)

In [ ]:
df_train

In [49]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    
    
    
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            if word_id is None:
                new_labels.append(-100)
            else:
                label = labels[word_id]
                
                new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            
            new_labels.append(label)
    
    return new_labels

In [50]:
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16165 sha256=6b87196f6b3ce693f79c91a6c8082b6e0c361a4c83a6806b4434527a17c88eb8
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [51]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.5 MB/s eta 0:00:00


In [52]:
import evaluate

metric = evaluate.load("seqeval")

In [ ]:
#tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
df_dev

In [147]:
b=tokenize_and_align_labels(n,tokenizer)
c=tokenize_and_align_labels(m,tokenizer)

In [ ]:
# for i in range(len(df_train)):
#     if len(df_train["tokens"].iloc[i,])!=len(df_train["tag"].iloc[i,]):
#         print(i)

In [110]:
n

,tokens,tag


In [111]:
label_names = m["tag"].explode().unique()

In [112]:
label_names

array(['O', 'B-Taxon', 'I-Taxon', 'B-GeographicalLocation',
       'I-GeographicalLocation', 'B-Habitat', 'I-Habitat', 'B-Person',
       'B-TemporalExpression', 'I-Person', 'I-TemporalExpression'],
      dtype=object)

In [119]:
#map={'Habitat':0, 'Taxon':1, 'Person':2, 'TemporalExpression':3,-100:-100
     
    
#}
map={'O':0, 'B-Taxon':1, 'I-Taxon':2, 'B-Person':3, 'B-TemporalExpression':4,
       'B-GeographicalLocation':5, 'I-GeographicalLocation':6,
       'I-TemporalExpression':7, 'B-Habitat':8, 'I-Habitat':9, 'I-Person':10,-100:-100
     
    
}

In [ ]:
map

In [149]:
b["labels"]=[[map[items] for items in label] for label in b["labels"]]

In [150]:
c["labels"]=[[map[items] for items in label] for label in c["labels"]]

In [105]:
label_names

array([], dtype=object)

In [ ]:
labels

In [ ]:
from seqeval.metrics import f1_score
from seqeval.metrics import accuracy_score
from seqeval.metrics import classification_report



In [122]:
def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
     }
#     f1=metric.f1_score(true_predictions,true_labels, average='micro')
#     pr=metric.precision_score(true_predictions,true_labels, average='micro')
#     ac=metric.accuracy_score(true_predictions,true_labels, average='micro')
#     r=metric.recall_score(true_predictions,true_labels, average='micro')
#     return {
#         "precision":pr,
#        "recall": r,
#        "f1": f1,
#        "accuracy": ac,
#     }
    #return classification_report(true_labels,true_predictions)
    
    

In [123]:
id2label = {i: label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

In [125]:
map

{'O': 0,
 'B-Taxon': 1,
 'I-Taxon': 2,
 'B-Person': 3,
 'B-TemporalExpression': 4,
 'B-GeographicalLocation': 5,
 'I-GeographicalLocation': 6,
 'I-TemporalExpression': 7,
 'B-Habitat': 8,
 'I-Habitat': 9,
 'I-Person': 10,
 -100: -100}

In [124]:
label2id

{'O': 0,
 'B-Taxon': 1,
 'I-Taxon': 2,
 'B-GeographicalLocation': 3,
 'I-GeographicalLocation': 4,
 'B-Habitat': 5,
 'I-Habitat': 6,
 'B-Person': 7,
 'B-TemporalExpression': 8,
 'I-Person': 9,
 'I-TemporalExpression': 10}

In [126]:
from transformers import AutoModelForTokenClassification


model =AutoModelForTokenClassification.from_pretrained(model_checkpoint,
  id2label=id2label,
 label2id=label2id,ignore_mismatched_sizes=True , from_tf=True
)

All TF 2.0 model weights were used when initializing BertForTokenClassification.

All the weights of BertForTokenClassification were initialized from the TF 2.0 model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BertForTokenClassification for predictions without further training.


In [127]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(model))

trainable model parameters: 108900107
all model parameters: 108900107
percentage of trainable model parameters: 100.00%


In [128]:
from huggingface_hub import notebook_login

notebook_login()


In [132]:
%pip install \
    rouge_score==0.1.2 \
    loralib==0.1.1 \
    peft==0.3.0 --quiet

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Note: you may need to restart the kernel to use updated packages.


In [158]:
#peft training
from peft import LoraConfig, get_peft_model, TaskType,PeftConfig

peft_config = LoraConfig(
    task_type=TaskType.TOKEN_CLS, inference_mode=False, r=16, lora_alpha=16, lora_dropout=0.1, bias="all"
)

In [134]:
import wandb

In [135]:
wandb.login()


wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [139]:
os.environ["WANDB_PROJECT"]="ner-peft"

# save your trained model checkpoint to wandb
os.environ["WANDB_LOG_MODEL"]="true"

# turn off watch to log faster
os.environ["WANDB_WATCH"]="false"

In [171]:
label2id

{'O': 0,
 'B-Taxon': 1,
 'I-Taxon': 2,
 'B-GeographicalLocation': 3,
 'I-GeographicalLocation': 4,
 'B-Habitat': 5,
 'I-Habitat': 6,
 'B-Person': 7,
 'B-TemporalExpression': 8,
 'I-Person': 9,
 'I-TemporalExpression': 10}

In [172]:
original_model=AutoModelForTokenClassification.from_pretrained("urbija/ner-bio-annotated-7-1",num_labels=11, id2label=id2label, label2id=label2id
)

In [173]:
peft_model = get_peft_model(original_model, 
                            peft_config)
print(print_number_of_trainable_model_parameters(peft_model))

trainable model parameters: 708886
all model parameters: 109498390
percentage of trainable model parameters: 0.65%


In [143]:
training_args = TrainingArguments(
    output_dir="bioner-peft",
    learning_rate=1e-3,
    
    num_train_epochs=1,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

In [174]:
trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=dataset,
    eval_dataset=dataset1,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.217550,0.800876,0.830748,0.815538,0.936964


TrainOutput(global_step=67, training_loss=0.10828386847652606, metrics={'train_runtime': 1250.2667, 'train_samples_per_second': 0.426, 'train_steps_per_second': 0.054, 'total_flos': 140262117844992.0, 'train_loss': 0.10828386847652606, 'epoch': 1.0})

In [179]:
peft_model.push_to_hub("urbija/bioner-peft")

adapter_model.bin:   0%|          | 0.00/2.85M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/urbija/bioner-peft/commit/edc586e4c291adc1b91c9e64fbbeb635fa0c2eee', commit_message='Upload model', commit_description='', oid='edc586e4c291adc1b91c9e64fbbeb635fa0c2eee', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
import tensorflow as tf

In [ ]:
label_names

In [ ]:
# h,t,g,p,t1=0,0,0,0,

In [ ]:
m.to_csv('/kaggle/working/dev.csv')

In [ ]:
n.to_csv('/kaggle/working/train.csv')

In [167]:
from peft import PeftModel, PeftConfig
import torch

In [180]:
peft_model_id = "urbija/bioner-peft"
config = PeftConfig.from_pretrained(peft_model_id)
inference_model = AutoModelForTokenClassification.from_pretrained(
    config.base_model_name_or_path, num_labels=11, id2label=id2label, label2id=label2id
)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
model = PeftModel.from_pretrained(inference_model, peft_model_id)

In [203]:
text="Luzon (Ilocos Norte, Apayao, Bontoc, Benguet, Nueva Vizcaya, Nueva Ecija, Bulacan, Zambales, Bataan, Laguna, Tayabas, Sorsogon), Mindoro, Palawan, Mindanao. On banks of ravines, slopes, about cliffs, etc, at low and medium, altitudes, ascending to 1,600 m. India through Malaya to New Guinea.Local names: Bugubui (Pamp) ; buybui (Ilk); eagadu (Bon); gatbo (Bik) ; lasa (Tag); tagadeu (Bon) , tagisa (Sub); tambu (Tag); talankaran (Ig)..DIGITARIA Scopoli, DIGITARIA CHINENSIS Nees in Hook. Local names: Timi (Bon); tumi (Bon); ualisibis (Iv). DIGITARIA CORYMBOSA (Roxb) comb. nov.Throughout the Philippines at low and medium altitudes in open waste places, old clearings, etc, usually common"


In [204]:
inputs = tokenizer(text, return_tensors="pt")

In [205]:
a=model(**inputs).logits

In [206]:
tokens = inputs.tokens()
predictions = torch.argmax(a,dim=2)

In [200]:
predictions

tensor([[0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 8, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 5, 5, 5, 5, 0, 5, 5, 5, 5, 5, 0, 5, 5, 6, 0, 0, 5, 5, 5, 5, 5, 5,
         0, 0, 0, 0, 0, 0, 1, 1, 5, 5, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2]])

In [207]:
for token, prediction in zip(tokens, predictions[0].numpy()):
    print((token, model.config.id2label[prediction]))


('[CLS]', 'O')
('luzon', 'B-Habitat')
('(', 'O')
('il', 'B-Habitat')
('##oco', 'B-Habitat')
('##s', 'B-Habitat')
('norte', 'I-Habitat')
(',', 'O')
('ap', 'B-Habitat')
('##aya', 'B-Habitat')
('##o', 'B-Habitat')
(',', 'I-Habitat')
('bon', 'B-Habitat')
('##to', 'B-Habitat')
('##c', 'B-Habitat')
(',', 'I-Habitat')
('ben', 'B-Habitat')
('##gue', 'B-Habitat')
('##t', 'B-Habitat')
(',', 'I-Habitat')
('nueva', 'B-Habitat')
('viz', 'I-Habitat')
('##ca', 'I-Habitat')
('##ya', 'I-Habitat')
(',', 'I-Habitat')
('nueva', 'B-Habitat')
('ec', 'I-Habitat')
('##ija', 'I-Habitat')
(',', 'O')
('bu', 'B-Habitat')
('##la', 'B-Habitat')
('##can', 'B-Habitat')
(',', 'O')
('za', 'B-Habitat')
('##mba', 'B-Habitat')
('##les', 'B-Habitat')
(',', 'I-Habitat')
('bat', 'B-Habitat')
('##aan', 'B-Habitat')
(',', 'I-Habitat')
('laguna', 'B-Habitat')
(',', 'I-Habitat')
('tay', 'B-Habitat')
('##abas', 'B-Habitat')
(',', 'O')
('so', 'B-Habitat')
('##rso', 'B-Habitat')
('##gon', 'B-Habitat')
(')', 'O')
(',', 'O')
('mind',